In [1]:
"""Importing Necessary Libraries"""

import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import xgboost as xgb
import warnings
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin
from sklearn.metrics import mean_squared_error

In [2]:
"""Importing already exported cleaned parquet file of the US_Traffic_2015 dataset"""

US_Traffic_2015=pd.read_parquet('.\\Data\\US_Traffic_2015.pqt')

## With the pool of independent variables available we can develop a predictive model to know apriori the volume of traffic in a particular day, in a particular area, in the jurisdiction of a particular station id, direction of travel and type of road. With the predictions available, Traffic Control can make sufficient planning for the ease of passage of vehicles

## Since we are predicting a continuous variable (Volume of Traffic), hence must to use a Regressor.

## There are lots of options among Regressors, namely Linear Regression, Decision Tree Regressor, Random Forest Regressor, Gradient Boosting Regressor or XGBoost Regressor.

## From the Bi-Variate analysis we observed that most of the independent variables are having a non-linear pattern with the target variables. Hence to develop a parametric model like Linear Regression, we have to perform lots of transformations to satisfy the assumptions.

## Under such circumstances usage of Machine Learning is a better idea. Among the Machine Learning algorithms XGBoost is the most popular one across the industry for its options of regularisation (which controls overfit) and parallel processing (faster model trainings). XGBoost algorithm also supports GPU acceleration, and hence for the above advantages the XGBoost Regressor was chosen for development of the predictive model.

## Since we have created 3 target variables namely, Traffic volumes of Non-Business Hours, Night-Time and Business Hours, we have chosen only one Target Variable for the predictive model development. Out of the 3, predicting traffic volumes during business hours (when there are a greater number of vehicles), will be more useful for practical usage

In [3]:
"""Identifying Objective or Target Variables"""

Target_Variables=[var for var in US_Traffic_2015.columns.tolist() if var.startswith('Traffic_Volume_')]

In [4]:
"""Retaining the Target Variable containing the information of Traffic Volume in Business Hours, and dropping the rest"""

Target_Variable=Target_Variables[-1]
Independent_Variables=list(set(US_Traffic_2015.columns.tolist())-set(Target_Variables))
Variables_to_Drop=Target_Variables[:2]
US_Traffic_2015.drop(Variables_to_Drop, axis=1, inplace=True)

## The entire dataset was split into 2 parts: a ~70% random sample for training and the remaining ~30% for In-Time Validation, hereafter will be called as Test.

## Under circumstances where there is absence of Out of Time dataset for model performance evaluation a K-fold cross validation is an industry standard practice.

## Since we have chosen XGBoost Regressor, we have to perform hyperparameter tuning to identify the most optimal model in order to attain stability and best performance in our Test dataset. For hyperparameter tuning there are multiple options, like Random Search, Manual Grid Search or Bayesian Optimisation to choose from. Bayesian Optimisation was chosen as the preferred method of hyperparameter tuning for its efficiency in moving towards the convergence with lesser number of experiments.

## Hyperparameter tuning is itself an expensive process and using K-Fold cross validation on top of it makes it even more complex, - hence K-Fold cross validation was avoided. Such an experiment requires cloud compute and can't be processed in a personal laptop.

In [5]:
"""Splitting the dataset into Training and Hold Out sets with a ratio of 70:30"""

X_Train, X_Test, y_Train, y_Test=train_test_split(US_Traffic_2015[Independent_Variables], US_Traffic_2015[Target_Variable], test_size=0.3, random_state=12345)

In [6]:
"""Converting the Train and Test datasets into XGBoost DMatrix"""

dtrain=xgb.DMatrix(data=X_Train, label=y_Train)
dtest=xgb.DMatrix(data=X_Test, label=y_Test)

## We have created a custom objective function for the Bayesian approach of Hyperparameter tuning:

## We want to maximise the Test dataset performance (here we have used adjusted R Squared as our evaluation metric), also we want to minimise overfit (Here we have defined overfit as the relative difference of adjusted R Squared between the Train dataset and the Test dataset. 

## In order to achieve this 2-fold optimisation here we have introduced a concept called Ideal Model. We have defined Ideal model as the one having ideal adjusted R Squared which is 1, and ideal overfit, which is 0 (No drop of Test performance from Train).

## In a 2D Cartesian system (Test Performance vs Overfit) the ideal model can be marked with the point (1,0). For any given hyperparameter we can plot the respective Test Performance and Overfit in the cartesian plane. The best model will be the one having minimum Euclidean Distance from the Ideal Model

## For all practical purposes we have limited the total number of HyperOpt trials to 100, and utilised GPU acceleration using the GeForce Gaming GPU available in the laptop, which is definitely faster than a Intel Dual Core CPU when parallel processing is taken into account.

In [7]:
"""Creating Helper Functions to run XGBoost using HyperOpt. Usage of each function has been explained below"""

"""Creating a function for computing Adjusted R Squared, which will be used as one of our model evaluation metrics"""

def Adjusted_R_Squared(Actuals, Predicted, nrows, ncols):
    df=pd.DataFrame({'Actuals':Actuals,'Predicted':Predicted})
    R_squared=1-(sum((df['Actuals']-df['Predicted'])**2)/sum((df['Actuals']-np.mean(df['Actuals']))**2))
    adj_R_squared=1-(1-R_squared)*((nrows-1)/(nrows-ncols-1))
    return(adj_R_squared)

"""The HyperOpt function with custom objective, which takes care of maximisation of Hold Out Performance and minimisation of
    overfit"""

def HyperOpt_Objective(space):
    warnings.filterwarnings(action='ignore')
    
    params={
        
        "learning_rate":space['learning_rate'], \
        "max_depth":int(space['max_depth']), \
        "gamma":space['gamma'], \
        "reg_lambda":space['reg_lambda'], \
        "reg_alpha":space['reg_alpha'], \
        "subsample":space['subsample'], \
        "colsample_bytree":space['colsample_bytree'], \
        "objective":'reg:squarederror', \
        "n_jobs":-1, \
        "tree_method":'gpu_hist', \
        "gpu_id":0, \
        "seed":12345
    }
    
    xgb_model=xgb.XGBRegressor(**params)
    xgb_model=xgb.train(params=xgb_model.get_xgb_params(), dtrain=dtrain, num_boost_round=5000, evals=[(dtrain, 'train'), (dtest, 'test')], early_stopping_rounds=10, verbose_eval=False)
    
    Train_Adjusted_R_Squared=Adjusted_R_Squared(y_Train.tolist(), xgb_model.predict(dtrain).tolist(), X_Train.shape[0], X_Train.shape[1])
    Test_Adjusted_R_Squared=Adjusted_R_Squared(y_Test.tolist(), xgb_model.predict(dtest).tolist(), X_Test.shape[0], X_Test.shape[1])
    
    if Train_Adjusted_R_Squared>Test_Adjusted_R_Squared:
        Euclidean_Distance_from_Ideal=np.sqrt(((1-Test_Adjusted_R_Squared)**2+(0-((Train_Adjusted_R_Squared-Test_Adjusted_R_Squared)/Train_Adjusted_R_Squared))**2))
    else:
        Euclidean_Distance_from_Ideal=np.sqrt(((1-Test_Adjusted_R_Squared)**2))

    return{'loss': Euclidean_Distance_from_Ideal, 'status': STATUS_OK, 'num_boost_round': xgb_model.best_ntree_limit, 'Train_Adjusted_R_Squared':Train_Adjusted_R_Squared, 'Test_Adjusted_R_Squared':Test_Adjusted_R_Squared}

In [8]:
"""Running XGBoost using HyperOpt with custom objective function"""

space = {
            'learning_rate': hp.quniform('learning_rate', 0.01, 0.1, 0.01),
            'max_depth': hp.quniform('max_depth', 1,10,1),
            'gamma': hp.quniform('gamma', 0,10,1),
            'reg_lambda': hp.quniform('reg_lambda', 0,10,1),
            'reg_alpha': hp.quniform('reg_alpha', 0,10,1),
            'subsample': hp.quniform('subsample', 0.1,1,0.1),
            'colsample_bytree': hp.quniform('colsample_bytree', 0.1,1,0.1)
        }

trials=Trials()
best=fmin(fn=HyperOpt_Objective, space=space, algo=tpe.suggest, max_evals=100, trials=trials)
print('Best model: ', best)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [11:20:58<00:00, 408.58s/trial, best loss: 0.05151237610261228]
Best model:  {'colsample_bytree': 0.5, 'gamma': 7.0, 'learning_rate': 0.04, 'max_depth': 9.0, 'reg_alpha': 0.0, 'reg_lambda': 7.0, 'subsample': 1.0}


In [9]:
"""Extracting the HyperOpt Log for conversion into a pandas dataframe"""

HyperOpt_Log=pd.DataFrame({})
for i in range(len(trials.trials)):
    Trial_Results=pd.DataFrame(trials.trials[i]['misc']['vals'])
    Trial_Results.insert(0,'Step',[trials.trials[i]['misc']['tid']])
    Trial_Results['num_boost_round']=[trials.trials[i]['result']['num_boost_round']]
    Trial_Results['loss']=[trials.trials[i]['result']['loss']]
    Trial_Results['Train_Adjusted_R_Squared']=[trials.trials[i]['result']['Train_Adjusted_R_Squared']]
    Trial_Results['Test_Adjusted_R_Squared']=[trials.trials[i]['result']['Test_Adjusted_R_Squared']]
    HyperOpt_Log=HyperOpt_Log.append(Trial_Results)

In [10]:
"""Exporting the HyperOpt Log"""

writer=pd.ExcelWriter('.\Output\HyperOpt_Log.xlsx', engine='xlsxwriter')
HyperOpt_Log.to_excel(writer, sheet_name='HyperOpt_Log', index=False)
for index, var in enumerate(HyperOpt_Log):
    max_length=max(HyperOpt_Log[var].astype(str).map(len).max(), len(var))+1
    writer.sheets['HyperOpt_Log'].set_column(index, index, max_length)    
writer.save()

In [11]:
"""Training the model with the best hyperparameter as per our custom objective function and HyperOpt"""

HyperOpt_Log=HyperOpt_Log.sort_values('loss').reset_index(drop=True)

params={

    "learning_rate":HyperOpt_Log.iloc[:1,1:9].to_dict()['learning_rate'][0], \
    "max_depth":int(HyperOpt_Log.iloc[:1,1:9].to_dict()['max_depth'][0]), \
    "gamma":HyperOpt_Log.iloc[:1,1:9].to_dict()['gamma'][0], \
    "reg_lambda":HyperOpt_Log.iloc[:1,1:9].to_dict()['reg_lambda'][0], \
    "reg_alpha":HyperOpt_Log.iloc[:1,1:9].to_dict()['reg_alpha'][0], \
    "subsample":HyperOpt_Log.iloc[:1,1:9].to_dict()['subsample'][0], \
    "colsample_bytree":HyperOpt_Log.iloc[:1,1:9].to_dict()['colsample_bytree'][0], \
    "objective":'reg:squarederror', \
    "n_jobs":-1, \
    "tree_method":'gpu_hist', \
    "gpu_id":0, \
    "seed":12345
}

xgb_model=xgb.XGBRegressor(**params)
xgb_model=xgb.train(params=xgb_model.get_xgb_params(), dtrain=dtrain, num_boost_round=int(HyperOpt_Log.iloc[:1,1:9].to_dict()['num_boost_round'][0]), evals=[(dtrain, 'train'), (dtest, 'test')], early_stopping_rounds=10, verbose_eval=False)

## Based on the best model chosen using the HyperOpt search, the training process was executed once more to have the final model with the best chosen hyperparameter combination.

## The Adjusted R Squared of the Train and Test datasets along with the Normalised Root Mean Squared Errors have been presented below

In [12]:
"""Evaluating the Train and Test performances based on the best model selection"""

print("Adjusted R Squared of Train dataset: "+str(Adjusted_R_Squared(y_Train.tolist(), xgb_model.predict(dtrain).tolist(), X_Train.shape[0], X_Train.shape[1])))
print("Adjusted R Squared of Test dataset: "+str(Adjusted_R_Squared(y_Test.tolist(), xgb_model.predict(dtest).tolist(), X_Test.shape[0], X_Test.shape[1])))
print("\n")
print("NRMSE of Train dataset: "+str(mean_squared_error(y_Train.tolist(), xgb_model.predict(dtrain).tolist(), squared=False)/y_Train.mean()))
print("NRMSE of Test dataset: "+str(mean_squared_error(y_Test.tolist(), xgb_model.predict(dtest).tolist(), squared=False)/y_Test.mean()))

Adjusted R Squared of Train dataset: 0.9436536321561113
Adjusted R Squared of Test dataset: 0.948497356140137


NRMSE of Train dataset: 0.3604653615246144
NRMSE of Test dataset: 0.34062225149390435


## The most significant variables of the model are also printed below, and we see that significant value has been generated by means of the feature engineering. The Weather data along with the Holiday information were utilised multiple times for creating the tree splits

In [13]:
"""Printing the top 15 most important features"""

Variable_Importance=pd.DataFrame(xgb_model.get_score(importance_type='weight').items())
Variable_Importance.columns=['Variable','Weight']
Variable_Importance=Variable_Importance.sort_values('Weight',ascending=[0]).reset_index(drop=True)
Variable_Importance.head(15)

,Variable,Weight
0,tavg,60316
1,day_of_week,59485
2,month_of_data,56036
3,day_of_data,54591
4,station_location_index,49466
5,lrs_location_point,38475
6,posted_signed_route_number_index,36913
7,fips_county_code,36597
8,lane_of_travel,33319
9,direction_of_travel,32968


In [14]:
"""Exporting the Variable Importance"""

writer=pd.ExcelWriter('.\Output\Variable_Importance.xlsx', engine='xlsxwriter')
Variable_Importance.to_excel(writer, sheet_name='Variable_Importance', index=False)
for index, var in enumerate(Variable_Importance):
    max_length=max(Variable_Importance[var].astype(str).map(len).max(), len(var))+1
    writer.sheets['Variable_Importance'].set_column(index, index, max_length)    
writer.save()